# 8. Beta

## 8.1 Estimating Beta

### 8.1.1 Description

***CAPM regression/One-factor market model regression***:
$$
r_{i,t} = \alpha _{i}+\beta _{i}MKT_{t}+\varepsilon _{i,t}
$$

| Data Frequencies | Estimation Period          | Minimum Valid Data points           |
| ---------------- | -------------------------- | ----------------------------------- |
| Daily Return     | 1, 3, 6, **12**, 24 months | 15, 50, 100, **200**, 450 daily obs |
| Monthly Retun    | 1, 2, 3, **5** years       | 10, 20, 24, **24** monthly obs      |

Note:
 
*Most common: (12, 200) & (5, 24)*

*Both estimated at the end of each month t.*

***Scholes and Willams (1997): account for nonsynchronous trading***:
$$
r_{i,t} = a_{i}+b_{i}^{-}MKT_{t-1}+e_{i,t}^{-} 
$$
$$
r_{i,t} = a_{i}+b_{i}MKT_{t}+e_{i,t}
$$
$$
r_{i,t} = a_{i}+b_{i}^{+}MKT_{t+1}+e_{i,t}^{+} 
$$
and define
$$
\beta _{i}^{SW}=\frac{\hat b_{i}^{-}+\hat b_{i}+\hat b_{i}^{+}}{1+2\rho}
$$
$\rho$: first-order serial correlation of $MKT$

| Data Frequencies | Estimation Period          | Minimum Valid Data points           |
| ---------------- | -------------------------- | ----------------------------------- |
| Daily Return     | 12 months, [t-11, t]       | 200 daily obs                       |

***Dimson(1979): infrequently traded***
$$
r_{i,t} = \alpha _{i}+\sum _{k=-5}^{k=5} b_{i}^{k}MKT_{t+k}+\varepsilon _{i,t}
$$
$$
\beta _{i}^{D}=\sum _{k=-5}^{k=5} \hat b_{i}^{k}
$$

| Data Frequencies | Estimation Period          | Minimum Valid Data points           |
| ---------------- | -------------------------- | ----------------------------------- |
| Daily Return     | 12 months, [t-11, t]       | 200 daily obs                       |

### 8.1.2 Codes

#### Monthly Sample Construction

In [1]:
# Import
import pandas as pd
import numpy as np
from numba import *
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 6) #set default figure size
from pandas.tseries.offsets import *
import pyreadstat

In [2]:
# Parameters
file_path = "E:\Study\Data\CRSP_Stock\Data\Source\CRSP_M.sas7bdat"
start_date = '1963-6-30'
end_date = '2012-12-31'

# Read Data
crsp, meta = pyreadstat.read_sas7bdat(file_path)
crsp.columns = crsp.columns.map(lambda x: x.lower())

# Define the Sample
# crsp_m['date']=pd.to_datetime(crsp_m['date'].astype(str))
crsp['date']=pd.to_datetime(crsp['date'], format='%Y-%m-%d')
crsp['jdate']=crsp['date'] + MonthEnd(0) # Line up date to be end of month
crsp = crsp[(crsp['jdate'] >= start_date) & (crsp['jdate'] <= end_date)]

# change variable format to int
crsp[['permco','permno']] = crsp[['permco','permno']].astype(int)

# Fama-French Monthly Factor Data
ff_m, meta = pyreadstat.read_sas7bdat("E:\Study\Data\CRSP_Stock\Data\Source\FF_Factor_M.sas7bdat")
ff_m.columns = ff_m.columns.map(lambda x: x.lower())
ff_m['dateff']=pd.to_datetime(ff_m['dateff'], format='%Y-%m-%d')
ff_m['jdate'] = ff_m['dateff'] + MonthEnd(0)
crsp = pd.merge(crsp, ff_m[['jdate', 'mktrf', 'rf']], how='left', on=['jdate'])

In [3]:
# Delisting Adjustment
dlstcd_list = [500, 520] + list(range(551,575)) + [580, 584]
dlstcd_in_list = crsp['dlstcd'].map(lambda x: x in dlstcd_list)
dlret_isna = crsp['dlret'].isna()
dlstcd_isna = crsp['dlstcd'].isna()
ret_isna = crsp['ret'].isna()
crsp.loc[(dlstcd_in_list) & (dlret_isna), 'dlret'] = -0.3
crsp.loc[(- dlstcd_in_list) & (- dlstcd_isna) & (dlret_isna), 'dlret'] = -1

crsp['retadj'] = crsp['ret']
crsp.loc[- crsp['dlret'].isna(), 'retadj'] = crsp.loc[- crsp['dlret'].isna(), 'dlret'] # can't use dlret_isna now

In [4]:
# Year and Month Variables
crsp['year'] = crsp['date'].dt.year
crsp['month'] = crsp['date'].dt.month

# Excess Stock Returns
crsp['ex_retadj'] = crsp['retadj'] - crsp['rf']
crsp['ex_ret'] = crsp['ret'] - crsp['rf']

# Market Values
crsp['me'] = crsp['shrout'] * crsp['altprc'].abs() / 1000 # measured in millions of dollars
# crsp['me'] = crsp['shrout'] * crsp['prc'].abs() / 1000

# Market Factor
crsp['ex_mkt'] = crsp['vwretd'] - crsp['rf']


# U.S.-based common stocks: sharecode ('SHRCD') = 10 or 11
crsp.query('shrcd == 10 or shrcd == 11', inplace=True)

In [5]:
crsp_ret_m = crsp[['permno', 'date', 'ex_ret', 'mktrf']].rename(columns={'jdate':'date', 'ex_ret':'ret', 'mktrf':'mkt_ret'})
crsp_ret_m.drop_duplicates(subset=['permno', 'date'], keep='first', inplace=True) # there are some duplicates in CRSP !!!
crsp_ret_m

,permno,date,ret,mkt_ret
1,10000,1986-01-31,NaN,0.0065
2,10000,1986-02-28,-0.262443,0.0713
3,10000,1986-03-31,0.359385,0.0488
4,10000,1986-04-30,-0.103792,-0.0131
5,10000,1986-05-30,-0.227556,0.0462
...,...,...,...,...
3624474,93436,2012-08-31,0.040017,0.0255
3624475,93436,2012-09-28,0.026548,0.0273
3624476,93436,2012-10-31,-0.039328,-0.0176
3624477,93436,2012-11-30,0.202115,0.0078


#### Rolling Betas Calculation

In [6]:
# Beta Function
def calc_beta(df, min_periods):

    # drop all rows with any NaN values
    # df = df[~np.isnan(df).any(axis=1)] # numba not support advanced index for array
    mask = np.isnan(df)
    m,n = mask.shape
    nan_rows = []
    for i in range(m):
        for j in range(n):
            if mask[i, j] == 1:
                nan_rows.append(i)
    all_rows = list(range(m))
    keep_rows = list(set(all_rows).difference(set(nan_rows)))
    df = df[np.array(keep_rows),:]
    
    # rebuild a contigous numpy array for faster reg
    x = np.ascontiguousarray(df[:,0:1]) # first column is the market
    x = np.concatenate((np.ones_like(x), x), axis=1)
    y = np.ascontiguousarray(df[:,1])

    if len(df) >= min_periods:
        b=(np.linalg.pinv(x.T @ x)) @ x.T @ y
        return b[0], b[1]
    else:
        return np.nan, np.nan

In [7]:
# Rolling Estimation
def rolling_beta(df, window, min_obs):
    # Set time index for rolling
    df = df.set_index('date')
    df.index = pd.DatetimeIndex(df.index)

    # Create an analogous dataframe to store the results
    beta = pd.DataFrame(df['permno'])
    beta['beta'] = np.nan
    
    # Divide the groups based on stocks, and do the rolling calculation
    grp = df.groupby('permno')
    for stock, sub_df in grp:
        sub2_df = sub_df[['mkt_ret', 'ret']].sort_index() 
        result = sub2_df.rolling(window, min_periods=min_obs, method="table").apply(calc_beta, raw=True, engine="numba", args=(min_obs,))
        beta.loc[beta.permno == stock, 'beta'] = result.values[:,1]
    
    beta.reset_index(inplace=True)
    return beta

In [8]:
years = [1, 2, 3, 5]
windows = ['365d', '730d', '1095d', '1825d']
min_obs = [10, 20, 24, 24]
beta_m = crsp_ret_m[['date', 'permno']]
for yr, win, min in zip(years, windows, min_obs):
    beta_estimation = rolling_beta(crsp_ret_m, window=win, min_obs=min)
    beta_estimation.rename(columns={'beta':'beta_Y' + str(yr)}, inplace=True)
    beta_m = pd.merge(beta_m, beta_estimation, on=['date', 'permno'])

In [11]:
beta_m

,date,permno,beta_Y1,beta_Y2,beta_Y3,beta_Y5
0,1986-01-31,10000,NaN,NaN,NaN,NaN
1,1986-02-28,10000,NaN,NaN,NaN,NaN
2,1986-03-31,10000,NaN,NaN,NaN,NaN
3,1986-04-30,10000,NaN,NaN,NaN,NaN
4,1986-05-30,10000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2936967,2012-08-31,93436,1.473857,0.726207,0.499603,0.499603
2936968,2012-09-28,93436,1.469184,0.723641,0.500030,0.500030
2936969,2012-10-31,93436,1.919929,0.813749,0.523509,0.523509
2936970,2012-11-30,93436,2.028233,0.858381,0.504068,0.504068


In [12]:
beta_m.groupby('date')[['beta_Y1', 'beta_Y2', 'beta_Y3', 'beta_Y5']].describe().mean()

beta_Y1  count    4422.331092
         mean        1.132334
         std         1.340506
         min        -9.612535
         25%         0.372230
         50%         1.008255
         75%         1.779519
         max        16.392266
beta_Y2  count    4049.157983
         mean        1.143246
         std         0.946544
         min        -4.872644
         25%         0.547421
         50%         1.039116
         75%         1.635061
         max         9.329123
beta_Y3  count    3927.284034
         mean        1.145705
         std         0.818250
         min        -3.539084
         25%         0.610800
         50%         1.051868
         75%         1.582904
         max         7.297382
beta_Y5  count    3963.284034
         mean        1.144300
         std         0.737112
         min        -3.437742
         25%         0.655072
         50%         1.059045
         75%         1.542751
         max         6.404072
dtype: float64